In [41]:
import pyspark
from pyspark.sql import SparkSession
import getpass
from pyspark.sql.functions import isnan, when, count, col
spark = SparkSession.builder.appName("BDA_DA1").getOrCreate()  
username = getpass.getuser()
print("Name: "+username+" 17MIS1022")

Name: Karthik 17MIS1022


In [31]:
df = spark.read.csv("adult.csv", header=True, inferSchema=True)

In [32]:
df.printSchema()
df.show()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- earnings: string (nullable = true)

+---+----------------+------+------------+-------------+--------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+--------+
|age|       workclass|fnlwgt|   education|education-num|marital-status|       occupation|  relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|earnings|

In [37]:
#printing the unique values of the column workclass
for x in ["workclass","education","occupation","relationship","race","sex","income"]:
    try:
        uniques=df.select(x).distinct().rdd.map(lambda r: r[0]).collect()    
        print(x,"with length",len(uniques),uniques)
    except:
        pass

education with length 16 ['10th', 'Masters', '5th-6th', 'Assoc-acdm', 'Assoc-voc', '7th-8th', '9th', 'HS-grad', 'Bachelors', '11th', '1st-4th', 'Preschool', '12th', 'Doctorate', 'Some-college', 'Prof-school']
occupation with length 15 ['Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Farming-fishing', 'Craft-repair', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Other-service', 'NaN', 'Tech-support', 'Machine-op-inspct', 'Armed-Forces', 'Adm-clerical']
relationship with length 6 ['Own-child', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative', 'Husband']
race with length 5 ['Other', 'Amer-Indian-Eskimo', 'White', 'Asian-Pac-Islander', 'Black']
sex with length 2 ['Female', 'Male']


In [33]:
# print the count of categories in the education column 
df.groupBy("education").count().sort("count",ascending=True).show()	

+------------+-----+
|   education|count|
+------------+-----+
|   Preschool|   51|
|     1st-4th|  168|
|     5th-6th|  333|
|   Doctorate|  413|
|        12th|  433|
|         9th|  514|
| Prof-school|  576|
|     7th-8th|  646|
|        10th|  933|
|  Assoc-acdm| 1067|
|        11th| 1175|
|   Assoc-voc| 1382|
|     Masters| 1723|
|   Bachelors| 5355|
|Some-college| 7291|
|     HS-grad|10501|
+------------+-----+



In [34]:
#Replace "?" with None. 
df = df.replace(['?'], "NaN") 


In [35]:
df.show()

+---+----------------+------+------------+-------------+--------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+--------+
|age|       workclass|fnlwgt|   education|education-num|marital-status|       occupation|  relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|earnings|
+---+----------------+------+------------+-------------+--------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+--------+
| 90|             NaN| 77053|     HS-grad|            9|       Widowed|              NaN| Not-in-family|White|Female|           0|        4356|            40| United-States|   <=50K|
| 82|         Private|132870|     HS-grad|            9|       Widowed|  Exec-managerial| Not-in-family|White|Female|           0|        4356|            18| United-States|   <=50K|
| 66|             NaN|186061|Some-college|           10|       Widowed|              

In [36]:
#count the number of null values in dataframe
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+--------+
|age|workclass|fnlwgt|education|education-num|marital-status|occupation|relationship|race|sex|capital-gain|capital-loss|hours-per-week|native-country|earnings|
+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+--------+
|  0|     1836|     0|        0|            0|             0|      1843|           0|   0|  0|           0|           0|             0|           583|       0|
+---+---------+------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+--------+

